In [88]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup
import time
import re
import csv
import pandas as pd
import unicodedata

In [99]:
BASE_URL = "https://iqga.me"
HEADERS = {"User-Agent": "Mozilla/5.0"}
SEASON_URLS = ["https://iqga.me/base/statistics/?season=2018/2019", "https://iqga.me/base/statistics/?season=2019/2020",
               "https://iqga.me/base/statistics/?season=2020/2021", "https://iqga.me/base/statistics/?season=2021/2022",
               "https://iqga.me/base/statistics/?season=2022/2023", "https://iqga.me/base/statistics/?season=2023/2024",
               "https://iqga.me/base/statistics/?season=2024/2025"]

csv_names = ["2018_2019.csv", "2019_2020.csv", "2020_2021.csv",
             "2021_2022.csv", "2022_2023.csv", "2023_2024.csv",
             "2024_2025.csv"]

In [100]:
def get_package_links(url):
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    package_section = soup.find("h3", string="Пакеты")
    if not package_section:
        return []
    
    package_list = package_section.find_next("ol")
    package_links = []
    
    for link in package_list.select("li h4 a"):
        href = link.get("href")
        name = link.text.strip()
        full_url = BASE_URL + href
        package_links.append((name, full_url))

    return package_links

def get_take_data_with_selenium(url):
    options = Options()
    options.add_argument("--headless") 
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    driver.get(url)
    time.sleep(5)

    page_source = driver.page_source
    driver.quit()

    matches = re.findall(r'<div id="bar-(\d+)">\s*<small>Взятие:\s*(\d+/\d+)\s*—\s*(\d+)%', page_source)

    take_data = {int(m[0]): (m[1], m[2]) for m in matches}

    return take_data

def parse_questions(url, package_name, take_data):
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    questions = []
    for idx, li in enumerate(soup.select("li[data-question]"), start=1):
        question_text = li.find_all("p")[1].text.strip()
        answer = li.select_one(".answer b").next_sibling.strip()
        credit = li.select_one(".answer b:-soup-contains('Зачет')").next_sibling.strip() if li.select_one(".answer b:-soup-contains('Зачет')") else ""
        comment = li.select_one(".answer p:nth-of-type(2)").text.strip() if len(li.select(".answer p")) > 1 else ""
        comment = comment.replace("Комментарий:", "").strip()
        source = li.select_one(".answer p:nth-of-type(3) a").get("href") if li.select_one(".answer p:nth-of-type(3) a") else ""
        authors = ", ".join([a.text.strip().replace("\n", "").replace("\t", "") for a in li.select(".answer p:nth-of-type(4)")])
        authors = authors.replace("Автор:", "").strip()

        take_fraction, take_percentage = take_data.get(idx, ("", ""))

        questions.append([package_name, idx, question_text, answer, credit, comment, source, authors, take_fraction, take_percentage])

    return questions

def save_to_csv(data, filename):
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Пакет", "№", "Вопрос", "Ответ", "Зачет", "Комментарий", "Источник", "Авторы", "Доля взятия", "Процент взятия"])
        writer.writerows(data)


In [ ]:
for i in range(7):

    print("Получаем список пакетов...")
    package_list = get_package_links(SEASON_URLS[i])

    all_data = []

    for package_name, package_url in package_list:
        print(f"Обрабатываем пакет: {package_name}")
    
        print("Получаем данные о взятии...")
        take_data = get_take_data_with_selenium(package_url)

        print("Парсим вопросы...")
        questions = parse_questions(package_url, package_name, take_data)

        all_data.extend(questions)

    save_to_csv(all_data, csv_names[i])

    print(f"Готово! Данные сохранены в {csv_names[i]}")

Получаем список пакетов...
Обрабатываем пакет: Умлаут Special: Сова в пабе 2019
Получаем данные о взятии...
Парсим вопросы...
Обрабатываем пакет: Лига Сибири. VI тур
Получаем данные о взятии...
Парсим вопросы...
Обрабатываем пакет: Лига Сибири. II тур
Получаем данные о взятии...
Парсим вопросы...
Обрабатываем пакет: Лига Сибири. III тур
Получаем данные о взятии...
Парсим вопросы...
Обрабатываем пакет: Лига Сибири. I тур
Получаем данные о взятии...
Парсим вопросы...
Обрабатываем пакет: Лига Сибири. IV тур
Получаем данные о взятии...
Парсим вопросы...
Обрабатываем пакет: Лига Сибири. V тур
Получаем данные о взятии...
Парсим вопросы...
💾 Сохраняем всё в CSV...
Готово! Данные сохранены в 2018_2019.csv
Получаем список пакетов...
Обрабатываем пакет: Синхрон-lite. Выпуск XXX
Получаем данные о взятии...
Парсим вопросы...
Обрабатываем пакет: Кубок Оливье 2020
Получаем данные о взятии...
Парсим вопросы...
Обрабатываем пакет: Балтийский берег. IV тур
Получаем данные о взятии...
Парсим вопросы...


In [114]:
df_list = []
for file in csv_names:
    df = pd.read_csv(file)
    df["Год"] = file.replace(".csv", "")
    df_list.append(df)

df = pd.concat(df_list, ignore_index=True)
df.insert(0, "id", range(1, len(df) + 1))
df.to_csv("merged_data.csv", index=False)

In [ ]:
def remove_accents(text: str) -> str:
    if not isinstance(text, str):
        return text
    
    result = []
    i = 0
    while i < len(text):
        char = text[i]
        
        if char.lower() in ('й', 'ё'):
            result.append(char)
            i += 1
            if i < len(text) and unicodedata.combining(text[i]):
                i += 1
        else:
            normalized = unicodedata.normalize('NFKD', char)
            for c in normalized:
                if not unicodedata.combining(c):
                    result.append(c)
            i += 1
    
    return ''.join(result)

In [115]:
file_path = "merged_data.csv"
df = pd.read_csv(file_path)

df_with_percentage = df.dropna(subset=["Процент взятия"])

df_with_material = df_with_percentage[df_with_percentage["Вопрос"].str.startswith("Раздаточный материал:")]
df_without_material = df_with_percentage[~df_with_percentage["Вопрос"].str.startswith("Раздаточный материал:")]

df_no_percentage = df[df["Процент взятия"].isna()]


df_without_material.to_csv("questions_without_material.csv", index=False)
df_with_material.to_csv("questions_with_material.csv", index=False)
df_no_percentage.to_csv("questions_no_percentage.csv", index=False)

In [116]:
df = pd.read_csv('questions_without_material.csv')
ids_to_remove = {3148, 610, 3162, 3471, 2791}


pattern = re.compile(r'\b(?:дублет|дуплет|блиц)\b', re.IGNORECASE)

filtered_df = df[
    ~df['id'].isin(ids_to_remove) & 
    ~df['Вопрос'].str.contains(pattern, na=False, regex=True)
]


filtered_df.to_csv('questions_without_material.csv', index=False)

In [ ]:
df = pd.read_csv('questions_without_material.csv')

df['Вопрос'] = df['Вопрос'].apply(remove_accents)
df['Ответ'] = df['Ответ'].apply(remove_accents)
df['Зачет'] = df['Зачет'].apply(remove_accents)

df["Знаменатель"] = df["Доля взятия"].str.split("/").str[1]
df["Знаменатель"] = pd.to_numeric(df["Знаменатель"], errors="coerce")

df = df[df["Знаменатель"] >= 100]
df = df.drop(columns=["Знаменатель"])
df.to_csv('questions_without_material.csv', index=False)
df.to_json('questions_without_material.json', orient='records', indent=2, force_ascii=False)

In [119]:
df.head()

,id,Пакет,№,Вопрос,Ответ,Зачет,Комментарий,Источник,Авторы,Доля взятия,Процент взятия,Год
0,1,Умлаут Special: Сова в пабе 2019,1.0,"В передаче Андрея Понкратова рассказывается, ч...",Носами,Нос,Приветствие касанием носа – традиция жителей с...,https://youtu.be/BW63sNTOD8Q?t=1513,"Андрей Кунилов (Мюнхен), Марина Парр (Мюнхен)",120/132,91.0,2018_2019
1,2,Умлаут Special: Сова в пабе 2019,2.0,Одна из сцен “Московского центра драматургии и...,выключать мобильные телефоны,"выключать сотовые телефоны, синонимичные ответы",перед началом спектакля голос по громкоговорит...,https://typical-moscow.ru/mama-cdr-istoriya-od...,"Андрей Кунилов (Мюнхен), Марина Парр (Мюнхен)",107/132,81.0,2018_2019
2,3,Умлаут Special: Сова в пабе 2019,3.0,"Статья ""National Geographic"" ""ПРОПУСК тела про...",пластика,точный ответ,"В фотопроекте Григория Шелухина ""Крохотная сме...",http://www.nat-geo.ru/planet/1190948-plastika-...,Борис Рабинович (Тель-Авив),93/132,70.0,2018_2019
3,4,Умлаут Special: Сова в пабе 2019,4.0,"Фантастическое животное демимаска, придуманное...",Нострадамус,Точный ответ,Особо неуязвимым демимаску делает способность ...,https://ru.wikipedia.org/wiki/Нострадамус,Борис Рабинович (Тель-Авив),40/132,30.0,2018_2019
4,5,Умлаут Special: Сова в пабе 2019,5.0,В постановке мюнхенского детского театра один ...,[вместо] мозгов,"[вместо] ума. Зачет по слову “мозг”, “ум”, “ра...",Речь о постановке Волшебника страны Оз. В сцен...,NaN,Максим Мозуль (Мюнхен),44/132,33.0,2018_2019
